In [2]:
import os

if "notebooks" in os.getcwd():
    %cd ..
from pathlib import Path

import matplotlib.pyplot as plt
import math
import pandas as pd
import numpy as np
import seaborn as sns
from src.utils.defines import INTERIM_DATA_DIR ,AUGMENTED_DATA_DIR, PROCESSED_DATA_DIR, RAW_DATA_DIR
import matplotlib.patches as mpatches

D:\SemEval-Edos


In [156]:
######################################################################################################
# REFORMAT OUTPUT FROM UNLABELED - task a
# task_a_unlabeled = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, "probs_task_a_GAB_FULL.csv"))
# unlabeled_data = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, "unlabeled_data_GAB_FULL.csv"))

task_a_unlabeled = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, "probs_task_a_reddit.csv"))
unlabeled_data = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, "unlabeled_data_500k_reddit.csv"))

In [157]:
# create final new data
merged_df = task_a_unlabeled.merge(unlabeled_data, left_index=True, right_index=True, how='inner')

# Set a confidence threshold for pseudo-labeling
confidence_threshold = 0.99
test = np.array(task_a_unlabeled)

# Get the most confident predictions and their indices
most_confident_indices = np.where(test.max(axis=1) > confidence_threshold)[0]
most_confident_preds = pd.DataFrame(test.argmax(axis=1)[most_confident_indices])

most_confident_preds.columns = ['target_a']

text_task_a = unlabeled_data.iloc[most_confident_indices].reset_index(drop=True)
text_task_a.columns = ['text']

final_task_a = pd.concat([text_task_a, most_confident_preds],axis=1)
final_task_a

,text,target_a
0,Who cares about how many pictures or w.e? If a...,0
1,her twitter name is yikes. lmao,0
2,I agree. It's like giving your kids a vote whe...,0
3,Or you could just have one night stands like t...,0
4,Plow through. This is the normal learning curv...,0
...,...,...
415883,Like how Stalin made the Ukranians “fear for t...,0
415884,Tbh im not sure of the details behind it. Look...,0
415885,Right... mine was super heavy and I’d often bl...,0
415886,An argument against me saying that the meaning...,0


In [158]:
dummy_values = np.random.rand(len(final_task_a))
df = final_task_a.copy()
df = df.assign(target_b=pd.Series(dummy_values))
df = df.assign(target_c=pd.Series(dummy_values))
df = df.assign(rewire_id=pd.Series(dummy_values))
df = df.reindex(columns=['rewire_id', 'text', 'target_a', 'target_b', 'target_c'])
df.to_csv("task_a_reddit_aug_99conf_FULL.csv", index=False)

In [106]:
# take only sexist data
sexist_new = merged_df.loc[merged_df['target_a'] == '1']
# sexist_new.to_csv(f'task_b_reddit_missing.csv', index=False)

sexist_new = sexist_new.drop(['target_a'],axis=1)
# sexist_new.to_csv("unlabeled_task_b_reddit_missing.csv", index=False)
######################################################################################################

In [59]:
df1 = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, "unlabeled_task_b_reddit.csv"))
df2 = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, "unlabeled_task_b_reddit_missing.csv"))

In [107]:
df1[df1['text'].isin(df2['text'])]

,text
0,Of course I lose respect for a woman if I fuck...
1,If only this foid could actually speak English...
3,She looks hot. probably very high maintenance ...
4,Take the condom out of the trash can and rip i...
5,"If it makes you feel any better, women don't e..."
...,...
40694,Yes women are way more privileged legally. Whi...
40695,Society CANNOT stand when men behave like wome...
40697,No one ever said it was OK. Most men are fairl...
40698,She needs an excuse to post about it. She is s...


In [25]:
reddit_all = pd.read_csv(os.path.join(RAW_DATA_DIR, "reddit_1M_unlabelled.csv"))
reddit_500k = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, "unlabeled_data_500k_reddit.csv"))

In [99]:
missing_rows = reddit_all[~reddit_all['text'].isin(reddit_500k['0'])]

In [104]:
reddit_500k[reddit_500k['0'].str.contains(missing_rows['text'][999995])]

,0


In [36]:
missing_rows.to_csv('missing_rows_reddit_500k.csv', header=None, index=None)

In [110]:
# REFORMAT OUTPUT FROM UNLABELED - task b
# task_b_labeled = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, "probs_task_b_GAB_FULL.csv"))
task_b_labeled = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, "probs_task_b_reddit.csv"))
# unlabeled_task_b_GAB = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, "unlabeled_data_GAB_FULL.csv"))
unlabeled_task_b_reddit = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, "unlabeled_task_b_reddit.csv"))
# Set a confidence threshold for pseudo-labeling
confidence_threshold = 0.95
test = np.array(task_b_labeled)

# Get the most confident predictions and their indices
most_confident_indices = np.where(test.max(axis=1) > confidence_threshold)[0]
most_confident_preds = pd.DataFrame(test.argmax(axis=1)[most_confident_indices])
most_confident_preds.columns = ['target_b']

text_task_b = unlabeled_task_b_reddit.iloc[most_confident_indices].reset_index(drop=True)

final_task_b = pd.concat([text_task_b, most_confident_preds],axis=1)
final_task_b

,text,target_b
0,Women shouldn’t have rights to falsely accuse ...,3
1,Turn her into a joke. Shame her intelligently....,0
2,Chad (and his buddy Tyrone) probably tag team ...,0
3,Come on.. Even women (FOR FUCK'S SAKE..) are c...,3
4,Actually feminists in the 70s penned a bill th...,3
...,...,...
2790,I’ve been reading a lot about trades from my f...,3
2791,"The first allows you to pay women more, menstr...",3
2792,If a woman has a kid knowing the father wants ...,3
2793,Yes women are way more privileged legally. Whi...,3


In [111]:
unique_values, counts = np.unique(most_confident_preds, return_counts=True)

for value, count in zip(unique_values, counts):
    print(f"{value}: {count}")

0: 668
1: 279
2: 239
3: 1609


In [46]:
dummy_values = np.random.rand(len(final_task_b))
df = final_task_b.copy()
df = df.assign(target_a=pd.Series(dummy_values))
df = df.assign(target_c=pd.Series(dummy_values))
df = df.assign(rewire_id=pd.Series(dummy_values))
df = df.reindex(columns=['rewire_id', '0', 'target_a', 'target_b', 'target_c'])
df.to_csv("task_b_GAB_aug_95conf_FULL.csv", index=False)

In [159]:
######################################################################################################
# REFORMAT OUTPUT FROM UNLABELED - task c
task_c_labeled = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, "probs_task_c_FULL_distilbert.csv"))
unlabeled_task_b_gab_FULL = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, "unlabeled_task_b_gab_FULL.csv"))
unlabeled_task_b_reddit = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, "unlabeled_task_b_reddit.csv"))

In [167]:
confidence_threshold = 0.70
test = np.array(task_c_labeled)

# Get the most confident predictions and their indices
most_confident_indices = np.where(test.max(axis=1) > confidence_threshold)[0]
most_confident_preds = pd.DataFrame(test.argmax(axis=1)[most_confident_indices])
most_confident_preds.columns = ['target_c']

In [168]:
unique_values, counts = np.unique(most_confident_preds, return_counts=True)

for value, count in zip(unique_values, counts):
    print(f"{value}: {count}")

1: 223
2: 1178
3: 1320
5: 9742
6: 137
10: 42


In [164]:
most_confident_preds

,target_c
0,5
1,5
2,5
3,5
4,5
...,...
859,5
860,5
861,5
862,5
